In [21]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [22]:
import mpld3
mpld3.enable_notebook()

In [23]:
import sys
sys.path.append('../../Utils/')

In [24]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 10

In [25]:
import tensorflow as tf 
import numpy as np 
import pandas as pd 
import nltk
import math
import random
from sklearn.utils import shuffle
from load_imdb_data import load_imdb_data
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score 
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np 
import matplotlib.pyplot as plt 
np.random.seed(0)
from readWikiData import *

In [26]:
sentences, word2idx, idx2word, original_sentence = get_wikipedia_data(n_files=1, n_vocab=10000)

In [27]:
sentences[0], original_sentence[0], sentences[1], original_sentence[1]

([1247, 8, 7, 236, 461, 11, 6706, 10000, 2630, 153, 15, 2879, 1279],
 'Anarchism is a political philosophy that advocates self-governed societies based on voluntary institutions',
 [52,
  16,
  92,
  334,
  9,
  10000,
  2630,
  118,
  110,
  1188,
  28,
  618,
  93,
  43,
  1306,
  9,
  1279,
  153,
  15,
  10000,
  381,
  4530],
 'These are often described as stateless societies, although several authors have defined them more specifically as institutions based on non-hierarchical free associations')

In [28]:
idx2word[1247], idx2word[8], idx2word[7], idx2word[236]

('anarchism', 'is', 'a', 'political')

#### Implement Encoder Layer 

In [29]:
vocab_size = len(idx2word)
hidden_size = 100

In [30]:
x_t = tf.placeholder(tf.int32, shape=[1])
h_t_1 = tf.zeros(shape=(1, hidden_size))

In [31]:
# r_t
W_r = tf.Variable(tf.random_uniform([vocab_size, hidden_size], -1.0, 1.0), name="W_r")
U_r = tf.Variable(tf.random_uniform([hidden_size, hidden_size], -1.0, 1.0), name="U_r")

# z_t 
W_z = tf.Variable(tf.random_uniform([vocab_size, hidden_size], -1.0, 1.0), name="W_z")
U_z = tf.Variable(tf.random_uniform([hidden_size, hidden_size], -1.0, 1.0), name="U_z")

# h_t`
W_x = tf.Variable(tf.random_uniform([vocab_size, hidden_size], -1.0, 1.0), name="W_x")
U = tf.Variable(tf.random_uniform([hidden_size, hidden_size], -1.0, 1.0), name="U_z")

In [32]:
r_t = tf.sigmoid(tf.nn.embedding_lookup(W_r, x_t) + tf.matmul(h_t_1, U_r))
z_t = tf.sigmoid(tf.nn.embedding_lookup(W_z, x_t) + tf.matmul(h_t_1, U_z))
h_tt = tf.tanh(tf.nn.embedding_lookup(W_x, x_t) + tf.matmul(r_t * h_t_1, U))
h_t = (1 - z_t)*h_t_1 + z_t*h_tt

In [33]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    for item in sentences[0]:
        output = sess.run([h_t], feed_dict={x_t:[item]})
        h_t_1 = output[-1][-1] 
    print output

[array([[ 0.08950575, -0.34388927, -0.09386601, -0.07920617, -0.34724224,
         0.004627  , -0.25099581, -0.18330878,  0.32205814,  0.11381489,
         0.10609717, -0.34478155, -0.25555316, -0.40183201, -0.32660121,
         0.29049969,  0.34218106, -0.16096675,  0.27709273, -0.34433508,
        -0.0330869 ,  0.26030418, -0.17714077, -0.27745989,  0.05411258,
        -0.04181414, -0.04167271,  0.3831557 ,  0.34315825, -0.16535401,
        -0.3470872 , -0.31899625,  0.04079741, -0.07716639, -0.12351626,
         0.27795142,  0.17825703,  0.37167796, -0.09916078, -0.4954519 ,
         0.10074893, -0.24270521,  0.47293916,  0.24984206, -0.13103037,
        -0.19470941,  0.47656071,  0.21887757, -0.29029882,  0.17899357,
        -0.21578616,  0.36319333,  0.3535828 , -0.30358949, -0.21026765,
        -0.29192999, -0.05463111,  0.20519921,  0.06344624,  0.21165419,
         0.2434991 ,  0.00180203, -0.1641418 ,  0.31749913, -0.26395839,
        -0.18859619,  0.20362677,  0.04450817, -0.

#### Implement Decoder Layer

In [34]:
x_t_1 = tf.placeholder(tf.int32, shape=[1])
h_td_1 = tf.zeros(shape=(1, hidden_size))

In [35]:
W_d_r = tf.Variable(tf.random_uniform([vocab_size, hidden_size], -1.0, 1.0), name="W_d_r")
U_d_r = tf.Variable(tf.random_uniform([hidden_size, hidden_size], -1.0, 1.0), name="U_d_r")
C_r = tf.Variable(tf.random_uniform([hidden_size, hidden_size], -1.0, 1.0), name="C_r")

W_d_z = tf.Variable(tf.random_uniform([vocab_size, hidden_size], -1.0, 1.0), name="W_d_z")
U_d_z = tf.Variable(tf.random_uniform([hidden_size, hidden_size], -1.0, 1.0), name="U_d_z")
C_z = tf.Variable(tf.random_uniform([hidden_size, hidden_size], -1.0, 1.0), name="C_z")

W_d = tf.Variable(tf.random_uniform([vocab_size, hidden_size], -1.0, 1.0), name="W_d")
U_d = tf.Variable(tf.random_uniform([hidden_size, hidden_size], -1.0, 1.0), name="U_d")
C = tf.Variable(tf.random_uniform([hidden_size, hidden_size], -1.0, 1.0), name="C")

In [36]:
r_t_d = tf.sigmoid(tf.nn.embedding_lookup(W_d_r, x_t_1) + tf.matmul(h_td_1, U_d_r) + tf.matmul(h_t, C_r))
z_t_d = tf.sigmoid(tf.nn.embedding_lookup(W_d_z, x_t_1) + tf.matmul(h_td_1, U_d_z) + tf.matmul(h_t, C_z))
h_tt_d = tf.tanh(tf.nn.embedding_lookup(W_d, x_t_1) + tf.matmul(r_t_d * h_td_1, U_d) + tf.matmul(h_t, C))
h_t_d = (1 - z_t_d)*h_td_1 + z_t_d*h_tt_d

In [37]:
V = tf.Variable(tf.random_uniform([vocab_size, hidden_size], -1.0, 1.0), name="V")
v_w = tf.nn.embedding_lookup(V, x_t_1)

In [38]:
loss = tf.log(tf.exp(tf.matmul(h_t_d, tf.transpose(v_w))))

In [39]:
loss = tf.reduce_mean(loss)

In [52]:
optimizer = tf.train.AdagradOptimizer(0.0001).minimize(loss)

In [58]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    for step in xrange(1001):
        epoch_error = 0.0 
    
        for item1, item2 in zip(sentences[:100][:-1], sentences[:100][1:]):
            # Encoding step 
            for item_x in item2:
                output_encoder = sess.run([h_t], feed_dict={x_t:[item_x]})
                h_t_1 = output_encoder[-1][-1] 

            h_t_1 = h_t_1.reshape(1, 100)

            # Decoding step 
            for item_y in item1:
                output_decoder, word_embeddings = sess.run([h_t_d, v_w], feed_dict={x_t_1:[item_y], \
                                                                                    h_t: h_t_1})
            
                h_t_1 = output_decoder
                
                l = sess.run(loss, feed_dict={h_t_d: output_decoder, v_w: word_embeddings})
                #if step % 10 == 0:
                #    print "Current loss : %s " %(l)
                epoch_error += l

        if step % 10 == 0:
            print "Error at epoch : ", step, " = ", epoch_error

Error at epoch :  0  =  165.031461875
Error at epoch :  10  =  165.031461875
Error at epoch :  20  =  165.031461875
Error at epoch :  30  =  165.031461875
Error at epoch :  40  =  165.031461875
Error at epoch :  50  =  165.031461875
Error at epoch :  60  =  165.031461875
Error at epoch :  70  =  165.031461875
Error at epoch :  80  =  165.031461875
Error at epoch :  90  =  165.031461875
Error at epoch :  100  =  165.031461875
Error at epoch :  110  =  165.031461875
Error at epoch :  120  =  165.031461875
Error at epoch :  130  =  165.031461875
Error at epoch :  140  =  165.031461875
Error at epoch :  150  =  165.031461875
Error at epoch :  160  =  165.031461875
Error at epoch :  170  =  165.031461875
Error at epoch :  180  =  165.031461875
Error at epoch :  190  =  165.031461875
Error at epoch :  200  =  165.031461875
Error at epoch :  210  =  165.031461875
Error at epoch :  220  =  165.031461875
Error at epoch :  230  =  165.031461875
Error at epoch :  240  =  165.031461875
Error at ep